In [1]:
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1# install StellarGraph if running on Google Colab
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
    

In [2]:
file_url= "E:\cons\pony\data\market\MarketSales_grouped_small.xlsx"
#file_url= "E:\cons\pony\data\market\MarketSales_grouped.xlsx"
grouped_purchased = pd.read_excel(file_url) # This may take a couple minutes
grouped_purchased.head()

,Unnamed: 0,CLIENTCODE,ITEMCODE,AMOUNT
0,0,1,1615,1.000
1,1,1,2733,1.000
2,2,1,2776,1.000
3,3,1,3780,1.000
4,4,1,3888,1.086


In [3]:
item_file_url = "E:\cons\pony\data\market\items.xlsx"
all_items = pd.read_excel(item_file_url) # This may take a couple minutes
all_items.head()
#grouped_cleaned = cleaned_retail.groupby(['CLIENTCODE', 'ITEMCODE']).sum().reset_index() # Group together

,ROWID,ITEMCODE,ITEMNAME,PRICE,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3
0,0,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ
1,1,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ
2,2,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ
3,3,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ
4,4,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ


In [13]:
#items = all_items.groupby(['ITEMCODE', 'ITEMNAME', 'PRICE', ]).sum().reset_index()
items = all_items.drop_duplicates(['ITEMCODE', 'ITEMNAME'])
items.head()

,ROWID,ITEMCODE,ITEMNAME,PRICE,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3
0,0,6.0,FLORMAR OJE,1.75,NaN,NaN,KOZMETİK,MAKYAJ MALZEMELERİ,MANİKUR PEDİKÜR ÜRÜNLERİ
58,58,7.0,EKMEK 250 GR,0.60,NaN,NaN,GIDA,UNLU MAMÜLLER,EKMEK
8888,8888,8.0,TOZ SEKER,2.65,NaN,NaN,İÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI
13979,13979,11.0,TRABZON&KOY EKMEGI,3.00,NaN,NaN,GIDA,UNLU MAMÜLLER,EKMEK
14687,14687,12.0,GILLETTE BLUE 2 PLUS BICAK,1.25,309,GILLETTE,KOZMETİK,TRAŞ MALZEMELERİ,TRAŞ ALETLERİ


In [14]:
items.to_excel('E:\cons\pony\data\market\items_unique.xlsx')

In [15]:
customer_file_url = "E:\cons\pony\data\market\customers.xlsx"
all_customers = pd.read_excel(customer_file_url) # This may take a couple minutes
all_customers.head()

,ROWID,CLIENTCODE,CLIENTNAME,CITY,REGION,LATITUDE,LONGITUDE,GENDER
0,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K
1,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K
2,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K
3,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K
4,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K


In [17]:
customers = all_customers.drop_duplicates(['CLIENTCODE'])
customers.head()

,ROWID,CLIENTCODE,CLIENTNAME,CITY,REGION,LATITUDE,LONGITUDE,GENDER
0,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K
7,NaN,456620,Nurgül ZÜLFİKAR,İstanbul,Marmara,41.0053,28.9770,K
14,NaN,275882,Aleyna ÇIKLASAĞIROĞLU,Adıyaman,Güneydoğu Anadolu,37.7648,38.2786,K
17,NaN,901974,Ege YOSUNLUKAYA,Kırklareli,Marmara,41.7333,27.2167,E
21,NaN,758893,İbrahim Halil LOKANTASI,Konya,İç Anadolu,37.8667,32.4833,E


In [18]:
#customers.to_excel('E:\cons\pony\data\market\customers_unique.xlsx') #gerek yok artık

In [29]:
unique_customer_file_url = "E:\cons\pony\data\market\customers_unique.xlsx"
unique_item_file_url = "E:\cons\pony\data\market\customers_unique.xlsx"

customers = pd.read_excel(unique_customer_file_url)
customers.head()

,Unnamed: 0,ROWID,CLIENTCODE,CLIENTNAME,CITY,REGION,LATITUDE,LONGITUDE,GENDER,AGE
0,0,NaN,6476,Hanım CANBULAT,Batman,Güneydoğu Anadolu,37.8812,41.1351,K,17
1,7,NaN,456620,Nurgül ZÜLFİKAR,İstanbul,Marmara,41.0053,28.9770,K,70
2,14,NaN,275882,Aleyna ÇIKLASAĞIROĞLU,Adıyaman,Güneydoğu Anadolu,37.7648,38.2786,K,77
3,17,NaN,901974,Ege YOSUNLUKAYA,Kırklareli,Marmara,41.7333,27.2167,E,64
4,21,NaN,758893,İbrahim Halil LOKANTASI,Konya,İç Anadolu,37.8667,32.4833,E,80


In [19]:
purchases_file_url = "E:\cons\pony\data\market\MarketSales_grouped_purchased.xlsx"
purchases = pd.read_excel(purchases_file_url) # This may take a couple minutes
purchases.head()

,Unnamed: 0,CLIENTCODE,ITEMCODE,ID,FICHENO,AMOUNT,PRICE,LINENETTOTAL,LINENET,BRANCHNR,LATITUDE,LONGITUDE
0,0,1,1615,188111,59977,1.000,0.85,0.85,0.79,1,37.0000,35.3213
1,1,1,2733,381019,104605,1.000,5.95,5.95,5.51,47,38.7312,35.4787
2,2,1,2776,189209,60106,1.000,0.75,0.75,0.69,40,41.0053,28.9770
3,3,1,3780,163899,54511,1.000,0.50,0.50,0.46,63,40.9839,37.8764
4,4,1,3888,188115,59977,1.086,11.95,12.98,12.02,1,37.0000,35.3213


In [30]:
edges = purchases[['ID', 'CLIENTCODE', 'ITEMCODE', 'AMOUNT']]
edges.head()
#edges.to_excel('E:\cons\pony\data\market\edges.xlsx')

KeyError: 'ITEMCODE'